In [1]:
import pandas as pd
import json
import pulp

# === Данные ===
signals_df = pd.read_csv("signals_current.csv")
flows_df = pd.read_csv("flows_peak.csv")
with open("constraints.json") as f:
    constraints = json.load(f)

NUM_INTERSECTIONS = signals_df.shape[0]
lost_time = 2 * constraints["lost_time_sec_per_phase"]

# === Модель ===
model = pulp.LpProblem("TrafficLightOptimization", pulp.LpMinimize)

# === Переменные для каждого перекрёстка ===
green_main = {i+1: pulp.LpVariable(f"green_main_{i+1}", lowBound=constraints["min_green_sec"], upBound=signals_df.iloc[i]["cycle_sec"]-constraints["min_green_sec"]-lost_time, cat='Integer') for i in range(NUM_INTERSECTIONS)}
green_sec = {i+1: pulp.LpVariable(f"green_sec_{i+1}", lowBound=constraints["min_green_sec"], upBound=signals_df.iloc[i]["cycle_sec"]-constraints["min_green_sec"]-lost_time, cat='Integer') for i in range(NUM_INTERSECTIONS)}

# === Ограничения по фазам ===
for i in range(1, NUM_INTERSECTIONS+1):
    cycle = signals_df.loc[signals_df['intersection_id'] == i, 'cycle_sec'].values[0]
    model += green_main[i] + green_sec[i] + lost_time <= cycle
    model += green_main[i] >= constraints['min_green_sec']
    model += green_sec[i] >= constraints['min_green_sec']
    model += green_main[i] >= constraints['pedestrian_green_sec']  # для главной

# === Целевая функция (минимизация задержки) ===
total_delay = []
for inter_id in range(1, NUM_INTERSECTIONS+1):
    cycle = signals_df.loc[signals_df['intersection_id'] == inter_id, 'cycle_sec'].values[0]
    df = flows_df[flows_df["intersection_id"] == inter_id]
    for _, row in df.iterrows():
        is_main = row["approach"] in ["N", "S"]
        flow = row["intensity_veh_per_hr"]
        bus_share = row["bus_share"]
        green = green_main[inter_id] if is_main else green_sec[inter_id]
        weight = 1 + 2 * bus_share
        total_delay.append((cycle - green) * flow * weight)

model += pulp.lpSum(total_delay)

# === Решение ===
solver = pulp.PULP_CBC_CMD(msg=True)
model.solve(solver)

# === Вывод ===
print("Status:", pulp.LpStatus[model.status])
for i in range(1, NUM_INTERSECTIONS+1):
    print(f"Перекрёсток {i}: Главная {green_main[i].varValue:.0f} сек, Второстепенная {green_sec[i].varValue:.0f} сек")
print(f"Суммарная задержка: {pulp.value(model.objective):,.2f} машино-секунд")


Status: Optimal
Перекрёсток 1: Главная 97 сек, Второстепенная 15 сек
Перекрёсток 2: Главная 97 сек, Второстепенная 15 сек
Перекрёсток 3: Главная 97 сек, Второстепенная 15 сек
Суммарная задержка: 449,333.00 машино-секунд
